In [ ]:
import cv2
import numpy as np


id_card = cv2.imread("id_card.jpg")
signature = cv2.imread("signature.jpg")

# Define signature area --------
x_offset, y_offset = 2025, 1330
sig_width, sig_height = 400, 220

# Crop the signature region from ID card
roi = id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width]

# Convert ROI to grayscale to detect dark ink
gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

# Create mask for black ink (adjust range as needed)
mask_black = cv2.inRange(gray_roi, 0, 70)  # 0–70 catches black and near-black

# Replace black pixels with average background color (greenish ID card tone)
background_color = [200, 230, 210]  # soft mint-green background
roi[mask_black > 0] = background_color

# Put back the cleaned ROI into the original image
id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width] = roi

cv2.imwrite("id_card1.jpg", id_card)



In [ ]:
import cv2
import numpy as np

# Load ID card and signature
id_card = cv2.imread("id_card1.jpg")
signature = cv2.imread("signature.jpg")

# Remove old signature from ID card

x_offset = 1985
y_offset = 1325
sig_width = 400
sig_height = 230

# Option A: Blur old signature area to blend with background
blurred_area = cv2.GaussianBlur(id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width], (15, 15), 0)
id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width] = blurred_area

# Prepare signature mask

gray = cv2.cvtColor(signature, cv2.COLOR_BGR2GRAY)

# Threshold to extract ink
_, alpha = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Clean noise
kernel = np.ones((3, 3), np.uint8)
alpha = cv2.morphologyEx(alpha, cv2.MORPH_OPEN, kernel)
alpha = cv2.morphologyEx(alpha, cv2.MORPH_DILATE, kernel)

# Convert signature color to BLACK 

black_sig = np.zeros_like(signature)
black_sig[:, :] = (0, 0, 0)  # black ink
signature_cleaned = cv2.bitwise_and(black_sig, black_sig, mask=alpha)

# Add alpha channel
b, g, r = cv2.split(signature_cleaned)
signature_rgba = cv2.merge((b, g, r, alpha))

# Resize and place signature
signature_resized = cv2.resize(signature_rgba, (sig_width, sig_height))
alpha_resized = signature_resized[:, :, 3] / 255.0

# Region of Interest from ID card (already cleared above)
roi = id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width]

# Blend new black signature
for c in range(3):
    roi[:, :, c] = (1 - alpha_resized) * roi[:, :, c] + alpha_resized * signature_resized[:, :, c]

# Put back
id_card[y_offset:y_offset+sig_height, x_offset:x_offset+sig_width] = roi


cv2.imwrite("id_card_with_black_signature.jpg", id_card)

